In [1]:
import torch
import pandas as pd
import numpy as np
import joblib
import os
import matplotlib.pyplot as plt
import seaborn as sns
import transformers
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


from transformers import BertTokenizer, AutoModel

from Preprocessing import preprocess_text

# os.chdir('C:/Users/LENOVO/GitHub/Jurnal-Clustering')

c:\Users\LENOVO\GitHub\Jurnal-Clustering\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class BertClassifier(nn.Module):
    def __init__(self, num_labels):
        super(BertClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained('bert-base-multilingual-cased')
        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 300),
            nn.ReLU(),
            nn.Linear(300, 100),
            nn.ReLU(),
            nn.Linear(100, 50),
            nn.ReLU(),
            nn.Linear(50, num_labels)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        x = outputs['last_hidden_state'][:, 0, :]
        x = self.classifier(x)
        return x

In [3]:
from torch.utils.data import Dataset, DataLoader

# Buat dataloader
class ArticleDataset(Dataset):
    def __init__(self, input_ids, attention_masks):
        self.input_ids = input_ids
        self.attention_masks = attention_masks

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx]
        }

    def __len__(self):
        return len(self.input_ids)


In [4]:
def tokenize_data(texts, tokenizer, max_length=256):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded_dict = tokenizer.encode_plus(
                            text,
                            add_special_tokens=True,
                            max_length=max_length,
                            pad_to_max_length=True,
                            return_attention_mask=True,
                            return_tensors='pt'
                       )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks

In [5]:
df = pd.read_csv('data/extracted_publication_journal_s2.csv')
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

df


,journal,title,abstract
0,23380403,Purwarupa Sistem Pemantau dan Peringatan Kadar...,"<jats:p>High growth in vehicle, contribute air..."
1,23380403,Implementasi Jaringan Syaraf Tiruan Perambatan...,<jats:p>Gold is one of commodities investment ...
2,23380403,Perancangan Jaringan Sensor Terdistribusi untu...,<jats:p>This paper contains distributed sensor...
3,23380403,Analisis Perbandingan Unjuk Kerja Sistem Penye...,<jats:p>The development of internet technology...
4,23380403,Pembuatan Aplikasi Mobile Learning sebagai Sar...,<jats:p>Mobile learning is the intersection of...
...,...,...,...
239319,28093216,Analysis of literacy-based learning media prod...,<jats:p>Literacy is an important skill that ev...
239320,28093216,Implementation of STEAM-based digital learning...,<jats:p>The lack of optimal empowerment of eff...
239321,28093216,Web-CMS based as teaching materials to improve...,<jats:p>Technology advances and the profusion ...
239322,28093216,SW-Blend: Student worksheets for blended learn...,<jats:p>Blended learning is learning that is a...


In [6]:
df.journal.nunique()

958

In [7]:
df.journal.unique()

array(['23380403', '25023993', '24775665', '25802364', '25496409',
       '27229696', '2407263X', '26148935', '24778524', '25414054',
       '26144735', '25498290', '25278827', '26563193', '26562804',
       '25795562', '26158558', '24430390', '25023489', '25497669',
       '25031481', '24609927', '24424730', '24076015', '25286579',
       '25286811', '26211378', '27234975', '24424226', '25498959',
       '26568810', '25491040', '26158582', '23021098', '24425249',
       '26569779', '24069280', '25026216', '25021907', '25287001',
       '25033360', '2581-2912', '25812912', '26543877', '2615840X',
       '25284339', '25279165', '26145863', '25492608', '26224372',
       '26232308', '2528584X', '25412590', '25490079', '26205092',
       '25026267', '2527628X', '23026332', '25798553', '26204053',
       '25487892', '25489364', '24771503', '26146908', '24769053',
       '26227371', '25485024', '25802070', '25495720', '24611433',
       '24430994', '2581236X', '25987771', '24607258', '25796

In [8]:
cwd = os.getcwd()

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertClassifier(12)
model.load_state_dict(torch.load('model/finetuned_model_sinta_translated.pt'))

# Membuat dataloader
batch_size = 32


In [9]:
for jour in df.journal.unique():

    journal_type = 'sinta_new_data_s2'

    # Assuming 'jurnal_id' is a variable containing the directory name
    file_path = os.path.join('src', journal_type, jour)

    if not os.path.exists(file_path):
        os.mkdir(file_path)

        data = df[df['journal'] == jour]
        data['data_cleaned'] = data['title'] + data['abstract'] 
        data['data_cleaned'].apply(lambda x : preprocess_text(x))
        X = list(data['data_cleaned'])

        input_ids, attention_masks = tokenize_data(X, tokenizer)
        dataset = ArticleDataset(input_ids, attention_masks)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

        device = 'cpu'
        if torch.cuda.is_available() :
            device = 'cuda'

        model.to(device)

        # Set model ke mode evaluasi (non-training)
        model.eval()

        # Embedding
        embeddings = []

        with torch.no_grad():
            for batch in dataloader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)

                outputs = model(input_ids, attention_mask=attention_mask)
                embeddings.append(outputs.cpu().numpy())

        embeddings = np.concatenate(embeddings, axis=0)

        # Mengubah array embeddings menjadi matriks dua dimensi
        X = embeddings.reshape(embeddings.shape[0], -1)

        pca = PCA(n_components=2, random_state=0)
        X = pca.fit_transform(X)

        # Perform KMeans clustering
        num_clusters = 1
        kmeans = KMeans(n_clusters=num_clusters, random_state=0, max_iter=1000)
        kmeans.fit(X)

        # Assign each journal to its cluster
        cluster_labels = kmeans.labels_

        # Mendapatkan koordinat pusat cluster
        centroid = kmeans.cluster_centers_

        # Menghitung jarak antara setiap titik data dengan centroid
        jarak_ke_centroid = np.sqrt(np.sum((X - centroid)**2, axis=1))

        # Menentukan batas jarak yang dianggap sebagai "outscoop"
        outscoop_threshold = np.mean(jarak_ke_centroid) + 2 * np.std(jarak_ke_centroid)

        # Memisahkan data yang masih masuk dalam "scoop" dan "outscoop"
        scoop_data = X[jarak_ke_centroid <= outscoop_threshold]
        outscoop_data = X[jarak_ke_centroid > outscoop_threshold]

        scoop_labels = np.ones(len(X))
        scoop_labels[jarak_ke_centroid > outscoop_threshold] = -1

        filename_kmeans = f"{file_path}/{jour}_kmeans.pkl"
        joblib.dump(kmeans, filename_kmeans)

        np.save(f"{file_path}/{jour}_threshold.npy", outscoop_threshold)
        np.save(f"{file_path}/{jour}_pca_data.npy", X)
        np.save(f"{file_path}/{jour}_bert_data.npy", embeddings.reshape(embeddings.shape[0], -1))

        df_res = pd.DataFrame({'Data': data['data_cleaned'],
                   'Label': scoop_labels})

        inScoop_df = df_res[df_res['Label'] == 1]
        outScoop_df = df_res[df_res['Label'] == -1]

        df_res.to_csv(f'{file_path}/{jour}_data_jurnal.csv')
        inScoop_df.to_csv(f'{file_path}/{jour}_inscoop_data_jurnal.csv')
        outScoop_df.to_csv(f'{file_path}/{jour}_outscoop_data_jurnal.csv')

        print("Data sebaran PCA {} telah disimpan.".format(jour))
    else:
        pass

    


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_2376\1070334198.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['data_cleaned'] = data['title'] + data['abstract']
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\LENOVO\GitHub\Jurnal-Clustering\.venv\lib\site-packages\transformers\tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version,

Data sebaran PCA 26143135 telah disimpan.


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_2376\1070334198.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['data_cleaned'] = data['title'] + data['abstract']
c:\Users\LENOVO\GitHub\Jurnal-Clustering\.venv\lib\site-packages\transformers\tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Data sebaran PCA 2540847X telah disimpan.


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_2376\1070334198.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['data_cleaned'] = data['title'] + data['abstract']
c:\Users\LENOVO\GitHub\Jurnal-Clustering\.venv\lib\site-packages\transformers\tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


...

Data sebaran PCA 23558520 telah disimpan.


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_2376\1070334198.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['data_cleaned'] = data['title'] + data['abstract']
c:\Users\LENOVO\GitHub\Jurnal-Clustering\.venv\lib\site-packages\transformers\tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Data sebaran PCA 2775202X telah disimpan.


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_2376\1070334198.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['data_cleaned'] = data['title'] + data['abstract']
c:\Users\LENOVO\GitHub\Jurnal-Clustering\.venv\lib\site-packages\transformers\tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Data sebaran PCA 28093216 telah disimpan.


In [10]:
# data = df[df['journal'] == '25024760']
# data['data_cleaned'] = data['title'] + data['abstract'] 
# data['data_cleaned'].apply(lambda x : preprocess_text(x))
# X = list(data['data_cleaned'])

# input_ids, attention_masks = tokenize_data(X, tokenizer)
# dataset = ArticleDataset(input_ids, attention_masks)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [11]:
# device = 'cpu'
# if torch.cuda.is_available() :
#     device = 'cuda'

# model.to(device)

# # Set model ke mode evaluasi (non-training)
# model.eval()

# # Embedding
# embeddings = []

# with torch.no_grad():
#     for batch in dataloader:
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)

#         outputs = model(input_ids, attention_mask=attention_mask)
#         embeddings.append(outputs.cpu().numpy())

# embeddings = np.concatenate(embeddings, axis=0)

In [12]:




# # Mengubah array embeddings menjadi matriks dua dimensi
# X = embeddings.reshape(embeddings.shape[0], -1)

# pca = PCA(n_components=2, random_state=0)
# X = pca.fit_transform(X)

# # Perform KMeans clustering
# num_clusters = 1
# kmeans = KMeans(n_clusters=num_clusters, random_state=0, max_iter=1000)
# kmeans.fit(X)

# # Assign each journal to its cluster
# cluster_labels = kmeans.labels_

# # Mendapatkan koordinat pusat cluster
# centroid = kmeans.cluster_centers_

# # Menghitung jarak antara setiap titik data dengan centroid
# jarak_ke_centroid = np.sqrt(np.sum((X - centroid)**2, axis=1))

# # Menentukan batas jarak yang dianggap sebagai "outscoop"
# outscoop_threshold = np.mean(jarak_ke_centroid) + 2 * np.std(jarak_ke_centroid)

# # Memisahkan data yang masih masuk dalam "scoop" dan "outscoop"
# scoop_data = X[jarak_ke_centroid <= outscoop_threshold]
# outscoop_data = X[jarak_ke_centroid > outscoop_threshold]

# scoop_labels = np.ones(len(X))
# scoop_labels[jarak_ke_centroid > outscoop_threshold] = -1

# filename_kmeans = f"{file_path}/{jour}_kmeans.pkl"
# joblib.dump(kmeans, filename_kmeans)

# np.save(f"{file_path}/{jour}_threshold.npy", outscoop_threshold)
# np.save(f"{file_path}/{jour}_pca_data.npy", X)
# np.save(f"{file_path}/{jour}_bert_data.npy", embeddings.reshape(embeddings.shape[0], -1))

# df_res = pd.DataFrame({'Data': data['data_cleaned'],
#             'Label': scoop_labels})

# inScoop_df = df_res[df_res['Label'] == 1]
# outScoop_df = df_res[df_res['Label'] == -1]

# df_res.to_csv(f'{file_path}/{jour}_data_jurnal.csv')
# inScoop_df.to_csv(f'{file_path}/{jour}_inscoop_data_jurnal.csv')
# outScoop_df.to_csv(f'{file_path}/{jour}_outscoop_data_jurnal.csv')
